In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

import regex 
import re, nltk
import seaborn as sns
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
wordnet_lemmatizer = WordNetLemmatizer()
from wordcloud import WordCloud, STOPWORDS



In [ ]:
review_data = pd.read_csv('olist_order_reviews_dataset.csv')
review_data.head()

In [ ]:
review_data.isnull().sum()

In [ ]:
review_data = review_data.drop(['review_comment_title'],axis=1)


In [ ]:
review_data1 = review_data.dropna()
review_data1.reset_index(drop=True)

In [ ]:
review_data1.head()

In [ ]:
reviews = []
stop_words = set(stopwords.words('portuguese'))


for words in df['review_comment_message']:
    only_letters = re.sub("[^a-zA-Z]", " ",words)
    tokens = nltk.word_tokenize(only_letters) #tokenize the sentences
    lower_case = [l.lower() for l in tokens] #convert all letters to lower case
    filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) #Remove stopwords from the comments
    lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result] #lemmatizes the words i.e convert similar words to their
    # base form while still considering the context in which the words are used 
    
    reviews.append(' '.join(lemmas))

In [ ]:
#Let's use worldcloud to visualize the messages
unique_string=(" ").join(reviews)
wordcloud = WordCloud(width = 2000, height = 1000,background_color='white').generate(unique_string)
plt.figure(figsize=(20,12))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
co = CountVectorizer(ngram_range=(1,1))
counts = co.fit_transform(reviews)
important_words = pd.DataFrame(counts.sum(axis=0),columns=co.get_feature_names()).T.sort_values(0,ascending=False).head(50)

In [ ]:
important_words=important_words.reset_index()


In [ ]:
important_words.head()

In [ ]:
important_words['index'].head()

In [ ]:
from translate import Translator
translator= Translator(from_lang="portuguese",to_lang="english")


In [ ]:
important_words['New'] = important_words['index'].apply(translator.translate)

In [ ]:
important_words

In [ ]:
co = CountVectorizer(ngram_range=(2,2))
counts = co.fit_transform(reviews)
important_biagrams = pd.DataFrame(counts.sum(axis=0),columns=co.get_feature_names()).T.sort_values(0,ascending=False).head(50)

In [ ]:
important_biagrams=important_biagrams.reset_index()

In [ ]:
important_biagrams['index'].head()

In [ ]:
important_biagrams['New'] = important_biagrams['index'].apply(translator.translate)

In [ ]:
important_biagrams

In [ ]:
from googletrans import Translator
translator = Translator()

In [ ]:
result = translator.translate("nao recebi",src ='pt', dest ='en')
print(result.text)

In [ ]:
co = CountVectorizer(ngram_range=(3,3))
counts = co.fit_transform(reviews)
important_triagrams = pd.DataFrame(counts.sum(axis=0),columns=co.get_feature_names()).T.sort_values(0,ascending=False).head(50)

In [ ]:
important_triagrams=important_triagrams.reset_index()

In [ ]:

important_triagrams['New'] = important_triagrams['index'].apply(translator.translate, src='es', dest='en').apply(getattr, args=('text',))



In [ ]:
important_triagrams

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation, NMF
vectorizer = CountVectorizer()
model = vectorizer.fit(df.review_comment_message)
docs = vectorizer.transform(df.review_comment_message)
lda = LatentDirichletAllocation(20)
lda.fit(docs)
def print_top_words(model, feature_names, n_top_words):
  for topic_idx, topic in enumerate(model.components_):
    message = "Topic #%d: " % topic_idx
    message += " ".join([(feature_names[i])
    for i in topic.argsort()[:-n_top_words - 1:-1]])
    print(message)
  print()
print_top_words(lda,vectorizer.get_feature_names(),10)

In [ ]:
df['topic']=lda.transform(docs).argmax(axis=1)
df.topic.value_counts(normalize=True).plot.bar()

In [ ]:
plt.figure(figsize = (14,6))
sns.countplot(review_data1['review_score'], color= 'red')

In [ ]:
plt.figure(figsize = (14,6))
sns.countplot(review_data['review_score'], color= 'red')